<a href="https://colab.research.google.com/github/sidharthdk/BMS-Data-pre-processing-files/blob/main/calce_data_cleaning_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [11]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

sidharthdk_calce_dataset_battery_archive_path = kagglehub.dataset_download('sidharthdk/calce-dataset-battery-archive')

print('Data source import complete.')


Data source import complete.


In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Important: Run the Cleaning Pipeline First!

It appears that the main data cleaning pipeline (cell ID `-wKxPwqsHVTC`) has not been executed yet. This cell processes the raw data and saves the cleaned CSV files into the `CLEAN_DIR`.

To ensure that you download all the *cleaned* CSV files, please **run cell ID `-wKxPwqsHVTC` first**. After it successfully completes, you can then **re-run the download cell (cell ID `896d650e`)** to get the zip archive containing all your cleaned data.

In [13]:
import shutil
from google.colab import files

output_zip_name = 'cleaned_battery_data.zip'
shutil.make_archive(output_zip_name.replace('.zip', ''), 'zip', CLEAN_DIR)

print(f"Created {output_zip_name}. Downloading...")
files.download(output_zip_name)

Created cleaned_battery_data.zip. Downloading...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
# ============================================================
# CALCE / HNEI Battery Dataset Cleaning Pipeline (Kaggle)
# ============================================================

import os
import pandas as pd
import numpy as np

# ----------------------------
# PATHS (KAGGLE)
# ----------------------------
RAW_DIR = "sidharthdk/calce-dataset-battery-archive"      # Kaggle dataset root
CLEAN_DIR = "/kaggle/working/cleaned"

os.makedirs(CLEAN_DIR, exist_ok=True)

# ----------------------------
# CONFIG
# ----------------------------
CONFIG = {
    "voltage_min": 2.0,
    "voltage_max": 4.5,
    "interp_max_gap": 5,
    "drop_bad_cycles": False
}

# ----------------------------
# UTILITY FUNCTIONS
# ----------------------------
def standardize_columns(df):
    df.columns = (
        df.columns.str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("(", "", regex=False)
        .str.replace(")", "", regex=False)
    )
    return df


def enforce_numeric(df):
    for c in df.columns:
        if c != "cycle_index":
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df


def remove_empty_columns(df):
    return df.dropna(axis=1, how="all")


def drop_duplicates_early(df):
    return df.drop_duplicates()


def validate_columns(df):
    required = ["cycle_index", "test_time_s", "voltage_v", "current_a"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing critical columns: {missing}")


def handle_missing(df, config):
    df = df.sort_values(["cycle_index", "test_time_s"])

    signal_cols = [c for c in df.columns if c in ["voltage_v", "current_a",
                                                  "cell_temperature_c",
                                                  "environment_temperature_c"]]

    state_cols = [c for c in df.columns if "capacity" in c or "energy" in c]

    if signal_cols:
        df[signal_cols] = df[signal_cols].interpolate(
            method="linear",
            limit=config["interp_max_gap"],
            limit_direction="both"
        )

    if state_cols:
        df[state_cols] = df[state_cols].ffill()

    df = df.dropna()
    return df


def enforce_physics(df, config):
    if "voltage_v" in df.columns:
        df = df[(df["voltage_v"] >= config["voltage_min"]) &
                (df["voltage_v"] <= config["voltage_max"])]

    for col in ["charge_capacity_ah", "discharge_capacity_ah",
                "charge_energy_wh", "discharge_energy_wh"]:
        if col in df.columns:
            df = df[df[col] >= 0]

    return df


def remove_outliers_cyclewise(df):
    cleaned = []

    for _, cdf in df.groupby("cycle_index"):
        for col in ["charge_capacity_ah", "discharge_capacity_ah"]:
            if col in cdf.columns and len(cdf) > 4:
                Q1 = cdf[col].quantile(0.25)
                Q3 = cdf[col].quantile(0.75)
                IQR = Q3 - Q1
                cdf = cdf[(cdf[col] >= Q1 - 1.5 * IQR) &
                          (cdf[col] <= Q3 + 1.5 * IQR)]
        cleaned.append(cdf)

    return pd.concat(cleaned, ignore_index=True)


def verify_cycles(df, config):
    bad = (
        df.groupby("cycle_index")["test_time_s"]
        .apply(lambda x: not x.is_monotonic_increasing)
    )
    bad_cycles = bad[bad].index.tolist()

    if bad_cycles:
        if config["drop_bad_cycles"]:
            df = df[~df["cycle_index"].isin(bad_cycles)]
        else:
            raise ValueError(f"Non-monotonic time in cycles: {bad_cycles}")

    return df


# ----------------------------
# MAIN LOOP (ALL CSV FILES)
# ----------------------------
for root, _, files in os.walk(RAW_DIR):
    for fname in files:
        if not fname.lower().endswith(".csv"):
            continue

        path = os.path.join(root, fname)
        print(f"\n🔹 Processing: {fname}")

        try:
            df = pd.read_csv(path)

            print("  Shape before:", df.shape)

            df = standardize_columns(df)
            df = remove_empty_columns(df)
            df = enforce_numeric(df)
            df = drop_duplicates_early(df)
            validate_columns(df)

            df = handle_missing(df, CONFIG)
            df = enforce_physics(df, CONFIG)
            df = remove_outliers_cyclewise(df)
            df = verify_cycles(df, CONFIG)

            df = df.drop_duplicates()

            print("  Shape after:", df.shape)

            out_path = os.path.join(
                CLEAN_DIR, fname.replace(".csv", "_cleaned.csv")
            )
            df.to_csv(out_path, index=False)

            print(f"✅ Saved: {out_path}")

        except Exception as e:
            print(f"❌ Failed on {fname}: {e}")
